In [2]:
from agents import Agent, WebSearchTool, trace, gen_trace_id, Runner
from agents.model_settings import ModelSettings
from dataclasses import dataclass, field
from typing import List, Optional
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

In [3]:
load_dotenv(override=True)

True

In [4]:
instructions="""
    You are an expert in startup competitor research.
    Given a startup idea and target market, find upto or less than 5 competitors.
    Use the WebSearchTool to gather data.
    """

@dataclass
class Competitor:
    name: str = Field(description="The name of the competitor")
    description: str = Field(description="Short 2-3 sentence description of the product")
    website: Optional[str] = Field(website="Website of the product if present")
    strengths: List[str] = Field(description="List of strenghts which standout in the product")
    weaknesses: List[str] = Field(description="List of weaknesses in the product")
    pricing: Optional[str] = Field(description="Pricing of the product if present")
    funding: Optional[str] = Field(description="Funding info of the competitor if present")
    market_position: Optional[str] = Field(description="Market share / position")

@dataclass
class CompetitorAnalysisResult:
    industry: str
    competitors: List[Competitor] = Field(description="List of competitors for the given business idea")
    insights: Optional[str] = None


competitor_agent = Agent(
    name="competitor_analysis_agent",
    tools=[WebSearchTool(search_context_size="low")],
    instructions=instructions,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
    output_type=CompetitorAnalysisResult
)

In [5]:
message = "AI Journalling app"

with trace("competition_research"):
    competitor_data = await Runner.run(competitor_agent, message)
    print(competitor_data.final_output)

CompetitorAnalysisResult(industry='AI Journaling Apps', competitors=[Competitor(name='Mentalio AI Mood Journal', description='An AI-powered mood journal designed to assist individuals with depression, stress, and anxiety through clinically proven methods, including guided affirmations, diary entries, breathing exercises, and mental health assessments.', website='https://www.producthunt.com/products/ai-diary/alternatives', strengths=['Comprehensive mental health support features', 'Clinically proven techniques', 'User-friendly interface'], weaknesses=['Limited platform availability', 'May require internet access for full functionality'], pricing='Freemium model with in-app purchases', funding='Not publicly disclosed', market_position='Emerging player in the mental health app market'), Competitor(name='Introspect Journal', description='An AI-driven journaling app that prompts users with deep questions to guide their thoughts, aiming to enhance self-reflection and clarity.', website='http

In [7]:
instructions="""
You are MarketResearchAgent, an AI agent specialized in researching and validating a business idea’s market potential.
Your role is to gather accurate, up-to-date, and credible market data from reputable sources using the WebSearchTool, and compile it into a structured report.
Your findings should help business stakeholders understand industry conditions, opportunities, risks, and target customer segments.
"""

class MarketResearchOutput(BaseModel):
    market_size: str = Field(
        description="The estimated current market size, expressed in monetary terms or user base."
    )
    growth_rate: str = Field(
        description="The expected annual growth rate of the market, ideally with a percentage."
    )
    key_trends: List[str] = Field(
        description="A list of notable trends currently shaping the market."
    )
    major_players: List[str] = Field(
        description="A list of leading companies or entities in this market."
    )
    barriers_to_entry: List[str] = Field(
        description="A list of significant challenges or obstacles new entrants may face."
    )
    opportunities: List[str] = Field(
        description="A list of promising opportunities for growth or differentiation in this market."
    )
    threats: List[str] = Field(
        description="A list of potential risks or external factors that could negatively affect the market."
    )
# Agent 3: Market Research
market_agent = Agent(
    name="market_research_agent",
    tools=[WebSearchTool(search_context_size="low")],
    instructions=instructions,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
    output_type=MarketResearchOutput
)

In [8]:
message = "AI Journalling app"

with trace("market_research"):
    market_data = await Runner.run(market_agent, message)
    print(market_data.final_output)

market_size='The global AI market was valued at USD 279.22 billion in 2024 and is projected to reach USD 390.91 billion in 2025, with a compound annual growth rate (CAGR) of 35.9% through 2030, reaching USD 1.81 trillion. The AI journaling niche benefits from this growth, driven by rising demand for mental health and productivity tools.' growth_rate='35.9% CAGR through 2030' key_trends=['Integration of AI with journaling apps for personalized mental health insights.', 'Use of large language models (LLMs) like ChatGPT for diary analysis and feedback.', 'Emergence of AI plugins and APIs enabling easy integration into mobile and web apps.', 'Growing focus on data privacy and compliance in personal AI applications.', 'Increasing user interest in AI-assisted productivity and emotional well-being.'] major_players=['Day One (Automattic)', 'Journey (Two App Studio)', 'Penzu', 'Diarium', 'Momento'] barriers_to_entry=['Integration and data privacy challenges.', 'Competition from established play

In [22]:
instructions="""
You are the Financial Estimator Agent.
Your job is to take competitor analysis data and market research data,
and estimate the Total Addressable Market (TAM), Serviceable Available Market (SAM),
Serviceable Obtainable Market (SOM), and potential annual revenues.

Steps:
1. Parse and understand the provided competitor and market datasets.
2. Use reasonable market assumptions (growth rates, adoption %, pricing models).
3. Clearly explain the calculations and assumptions.
4. Return a structured output in the expected schema.
"""

class FinancialEstimationResult(BaseModel):
    TAM: float = Field(
        ...,
        description="Total Addressable Market in USD. Represents the total revenue opportunity available if the product captured 100% of the market."
    )
    SAM: float = Field(
        ...,
        description="Serviceable Available Market in USD. Represents the portion of the TAM that can be targeted based on the business's product scope and geographical reach."
    )
    SOM: float = Field(
        ...,
        description="Serviceable Obtainable Market in USD. Represents the realistic market share the business can capture given competition, pricing, and operational constraints."
    )
    estimated_annual_revenue: float = Field(
        ...,
        description="Projected yearly revenue in USD based on current market share assumptions and pricing models."
    )
    assumptions: str = Field(
        ...,
        description="Key assumptions used in the estimation, including growth rates, pricing strategy, adoption rates, and market conditions."
    )

financial_estimator_agent = Agent(
    name="financial_estimator_agent",
    tools=[],  # No external search tools, works with provided data
    instructions=instructions,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="none"),
    output_type=FinancialEstimationResult
)

In [ ]:
message = "AI Journalling app"

fin_inputs = {
    "competitor_data": competitor_data,
    "market_data": market_data
}

with trace("financial_estimator"):
    financial_data = await Runner.run(
        financial_estimator_agent,
        str(fin_inputs)
        )
    print(financial_data.final_output)

TAM=1810000000000.0 SAM=390910000000.0 SOM=39091000000.0 estimated_annual_revenue=3909100000.0 assumptions='Assuming a conservative market penetration of 10% of the SAM based on the competitive landscape and pricing strategies. This includes projected growth from increased consumer interest in AI-assisted mental health tools.'


In [26]:
INSTRUCTIONS = (
    "You are a skilled business analyst and professional writer. "
    "Given the outputs from competitor analysis, market analysis, and financial estimation agents, "
    "write a cohesive business report. "
    "The report should have a clear structure, professional tone, and flow logically. "
    "Avoid redundancy. "
    "Include headings, subheadings, and summary insights."
    "Format the output as a markdown report with collapsible <details> sections for competitor, market data and financial data. Convert all tabular or bullet data into proper markdown tables or lists before embedding them."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [27]:
message = "AI Journalling app"

writer_inputs = {
    "competitor_data": competitor_data,
    "market_data": market_data,
    "financial_estimation_data": financial_data
}

with trace("report_writer"):
    report = await Runner.run(
        writer_agent,
        str(writer_inputs)
        )
    print(report.final_output)

short_summary='The analysis of the AI journaling app market reveals a dynamic landscape with various competitors offering specialized features. The market is projected to grow significantly, with a total addressable market (TAM) estimated at $1.81 trillion by 2030, indicating substantial opportunities for innovation and market penetration.' markdown_report='# AI Journaling App Market Report\n\n## Table of Contents\n- [Background](#background)\n- [Competitor Analysis](#competitor-analysis)\n- [Market Analysis](#market-analysis)\n- [Financial Estimation](#financial-estimation)\n- [Conclusion](#conclusion)\n- [Follow-Up Questions](#follow-up-questions)\n\n---\n\n## Background\nThe AI journaling app sector has gained prominence due to growing interest in mental health tools and productivity solutions. This report evaluates the current market landscape, competitors, growth potential, and financial opportunities.\n\n---\n\n## Competitor Analysis\n<details>\n<summary>Click to Expand Competito

In [28]:
final: ReportData = report.final_output
display(Markdown(final.markdown_report))

# AI Journaling App Market Report

## Table of Contents
- [Background](#background)
- [Competitor Analysis](#competitor-analysis)
- [Market Analysis](#market-analysis)
- [Financial Estimation](#financial-estimation)
- [Conclusion](#conclusion)
- [Follow-Up Questions](#follow-up-questions)

---

## Background
The AI journaling app sector has gained prominence due to growing interest in mental health tools and productivity solutions. This report evaluates the current market landscape, competitors, growth potential, and financial opportunities.

---

## Competitor Analysis
<details>
<summary>Click to Expand Competitor Analysis</summary>

The following are notable competitors in the AI journaling app market:

| Competitor Name                       | Description                                                                                                         | Website                                                          | Strengths                                                  | Weaknesses                                            | Pricing                                   | Funding                     | Market Position                       |
|---------------------------------------|---------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------|------------------------------------------------------------|------------------------------------------------|------------------------------------------|----------------------------|------------------------------------|
| Mentalio AI Mood Journal              | An AI-powered mood journal for individuals dealing with anxiety and depression.                                     | [Mentalio](https://www.producthunt.com/products/ai-diary/alternatives) | Comprehensive mental health support                      | Limited platform availability                      | Freemium with in-app purchases         | Not disclosed               | Emerging player in the mental health app market |
| Introspect Journal                    | An AI-driven journaling app prompting users with deep questions for self-reflection.                                | [Introspect](https://www.producthunt.com/products/ai-diary/alternatives) | Deep, thought-provoking prompts                            | Limited features compared to competitors       | Freemium model                        | Not disclosed               | Niche product in the journaling app market |
| DreamEscape: AI Dream Journal         | AI-powered dream journal providing personalized insights for understanding dreams.                                   | [DreamEscape](https://www.producthunt.com/products/ai-diary/alternatives) | Specialized focus on dream analysis                         | Requires active user engagement for insights   | Freemium model                        | Not disclosed               | Specialized app in the dream analysis niche |
| Daylio                                | Diary and mood tracking app with statistical insights.                                                             | [Daylio](https://daylio.net/)                                      | Simple and intuitive interface                              | Limited AI integration                        | Freemium with in-app purchases         | Not disclosed               | Established app with a large user base |
| Apple’s Journal                       | A journaling app by Apple Inc., featuring writing suggestions and prompts.                                         | [Apple’s Journal](https://en.wikipedia.org/wiki/Journal_%28Apple%29) | Seamless integration with Apple ecosystem                   | Limited only to Apple devices, lacks some features | Free for iOS 17.2+                   | Developed by Apple Inc.   | Strong position due to brand recognition |

### Insights
The AI journaling market is characterized by diverse offerings tailored for mood tracking, self-reflection, and dream analysis. While long-standing apps like Daylio and Apple’s Journal hold strong positions, newer entrants focus on specialized features that cater to niche audiences. The opportunities for innovation remain vast as user needs evolve.

</details>

---

## Market Analysis
<details>
<summary>Click to Expand Market Analysis</summary>

### Market Size and Growth
- **Total Addressable Market (TAM)**: $1.81 trillion by 2030
- **Projected CAGR**: 35.9% from 2025 to 2030 

### Key Trends
- Integration of AI for personalized mental health insights.
- Use of large language models (LLMs) for diary analysis.
- Emergence of plugins and APIs for app integration.
- Growing emphasis on data privacy in personal applications.
- Increasing user interest in AI-facilitated emotional well-being.

### Major Players
- Day One (Automattic)
- Journey (Two App Studio)
- Penzu
- Diarium
- Momento

### Barriers to Entry
- Integration and data privacy challenges.
- Competition from established players.
- High development costs for AI features.

### Opportunities
- Development of AI features for mental health.
- Voice-to-text functionality for hands-free journaling.
- Subscription models with tiered pricing.

### Threats
- Intense competition from both established and new players.
- Rapid technological advancements impacting user preferences.
- Data privacy and regulatory issues.

</details>

---

## Financial Estimation
<details>
<summary>Click to Expand Financial Estimation</summary>

### Key Financial Metrics
- **Total Addressable Market (TAM)**: $1.81 trillion
- **Serviceable Available Market (SAM)**: $390.91 billion
- **Serviceable Obtainable Market (SOM)**: $39.09 billion
- **Estimated Annual Revenue**: $3.91 billion

### Assumptions
The calculations are based on a conservative market penetration of 10% of the SAM due to favorable market conditions, consumer trends, and existing competition. 

</details>

---

## Conclusion
The AI journaling app market offers significant opportunities driven by technological advancements and growing consumer interest in mental health solutions. Companies can leverage existing technologies to innovate and capture market share in this rapidly evolving sector.

---

## Follow Up Questions
- What specific technologies are driving the growth of AI in wellness apps?
- How do user preferences vary across different demographics?
- What strategies are competitors employing to enhance user engagement?